## XFOIL analysis with python UI

    Commend "LOAD"
            "GDES" switches the XFOIL to geometry design stage
            "CADD" adds corner points to satisfy user specified angle threshold.
            "PANEL" regenerates vertices
            "OPER" switches the program to aerodynamic analysis mode
            "ALFA 5" angle of attack = 5
        

In [71]:
import subprocess as sp
import shutil
import sys
import string

In [72]:
def script(airfoil, data_loc, AOT, Reynold):
    load_script = ['load '+airfoil+".txt",
            'MDES',
            'FILT',
            'EXEC',
            ' ',
            'PANE',
            'OPER',
            'ITER 100']
    for RE in Reynold:
        data_file_name = data_loc+airfoil+'_'+str(RE)+'.txt'
        constant_set = ['RE '+str(RE),
                        'VISC '+str(RE), 
                        'PACC', 
                        data_file_name,
                        ' ']
        alpha = ['ALFA '+str(a) for a in AOT]
        init = constant_set + alpha + ['PACC']
        load_script += init
    load_script += [' ', ' ', 'QUIT\n']
    return load_script

In [73]:
ps = sp.Popen(['xfoil.exe'],
              stdin=sp.PIPE,
              stdout=None,
              stderr=None,
              encoding='utf8')
res = ps.communicate('\n'.join(script('S813', 'data/', [0,1,2,3,4,5], [300000,400000,500000])))